# 🖥️ Nemo tutorial


One of the features of Parcels is that it can directly and natively work with `Field` data discretised on C-grids. These C grids are very popular in OGCMs, so velocity fields outputted by OGCMs are often provided on such grids, except if they have been firstly re-interpolated on a A grid.

More information about C-grid interpolation can be found in [Delandmeter et al., 2019](https://www.geosci-model-dev-discuss.net/gmd-2018-339/).
An example of such a discretisation is the NEMO model, which is one of the models for which we provide support in Parcels. Other models are CROCO, ROMS, and MITgcm.

```{note}
_How to know if your data is discretised on a C grid?_ The best way is to read the documentation that comes with the data. Alternatively, an easy check is to assess the coordinates of the U, V and W fields: 
- for an **A grid**, U, V and W are distributed on the same nodes, such that the coordinates are the same. 
- for a **C grid**, there is a shift of half a cell between the different variables.
```

## Curvilinear C-Grids

Parcels supports [curvilinear grids](https://www.nemo-ocean.eu/doc/node108.html) such as those used in the [NEMO models](https://www.nemo-ocean.eu/).

```{note}
TODO: make explicit how Parcels determines rotation
```

We will be using the example dataset `NemoCurvilinear_data`. These fields are a purely zonal flow on an aqua-planet (so zonal-velocity is 1 m s<sup>-1</sup> and meridional-velocity is 0 m s<sup>-1</sup> everywhere, and no land). However, because of the curvilinear grid, the `U` and `V` fields vary for the rotated grid cells north of 20N.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import parcels

We use the `parcels.convert.nemo_to_sgrid()` function to convert the NEMO curvilinear grid to a dataset that is SGrid-compliant. We then pass this new dataset into `parcels.FieldSet.from_sgrid_conventions()`. This function automatically detects the C-grid structure and sets up the grid accordingly.

In [ ]:
data_folder = parcels.download_example_dataset("NemoCurvilinear_data")
ds_fields = xr.open_mfdataset(
    data_folder.glob("*.nc4"),
    data_vars="minimal",
    coords="minimal",
    compat="override",
)

ds_coords = xr.open_dataset(data_folder / "mesh_mask.nc4", decode_times=False)
ds_fset = parcels.convert.nemo_to_sgrid(
    fields=dict(U=ds_fields["U"], V=ds_fields["V"]), coords=ds_coords
)

fieldset = parcels.FieldSet.from_sgrid_conventions(ds_fset)

And we can plot the `U` field.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
pc1 = fieldset.U.data.plot(cmap="viridis", ax=ax[0], vmin=0)
pc2 = ax[1].pcolormesh(
    fieldset.U.grid.lon,
    fieldset.U.grid.lat,
    fieldset.U.data[0, 0, :, :],
    vmin=0,
    vmax=1,
)
ax[1].set_ylabel("Latitude [deg N]")
ax[1].set_xlabel("Longitude [deg E]")
plt.colorbar(pc2, label="U", ax=ax[1])
plt.tight_layout()
plt.show()

As you see above, the `U` field indeed is 1 m s<sup>-1</sup> south of 20N, but varies with longitude and latitude north of that. We can confirm that Parcels will take care to rotate the `U` and `V` fields by doing a field evaluation at (60N, 50E):

In [ ]:
u, v = fieldset.UV.eval(np.array([0]), np.array([0]), np.array([60]), np.array([50]))
u *= 1852 * 60 * np.cos(np.deg2rad(60))  # convert from degrees s^-1 to m s^-1
v *= 1852 * 60  # convert from degrees s^-1 to m s
print(f"(u, v) = ({u[0]:.3f}, {v[0]:.3f})")
assert np.isclose(u, 1.0, atol=1e-3)
assert np.isclose(v, 0.0, atol=1e-3)

Now we can run particles as normal.

In [ ]:
npart = 20
lonp = 30 * np.ones(npart)
latp = np.linspace(-70, 88, npart)
runtime = np.timedelta64(40, "D")

pset = parcels.ParticleSet(fieldset, lon=lonp, lat=latp)
pfile = parcels.ParticleFile(
    store="output_curvilinear.zarr", outputdt=np.timedelta64(1, "D")
)

pset.execute(
    [parcels.kernels.AdvectionEE],
    runtime=runtime,
    dt=np.timedelta64(1, "D"),
    output_file=pfile,
)
np.testing.assert_allclose(pset.lat, latp, atol=1e-1)

And then we can plot these trajectories. As expected, all trajectories go exactly zonal and due to the curvature of the earth, ones at higher latitude move more degrees eastward (even though the distance in km is equal for all particles). These particles at high latitudes cross the antimeridian (180 deg E) and keep going east.

In [ ]:
ds = xr.open_zarr("output_curvilinear.zarr")

plt.plot(ds.lon.T, ds.lat.T, ".-")
plt.vlines(np.arange(-180, 901, 360), -90, 90, color="r", label="antimeridian")
plt.ylabel("Latitude [deg N]")
plt.xlabel("Longitude [deg E]")
plt.xticks(np.arange(-180, 901, 90))
plt.legend(loc="lower right")
plt.show()

### Handling longitude wrapping
If we want the `particles.lon` to stay within `[-180,180]` (or `[0,360]`), we can either do this in post-processing, or add a periodic boundary Kernel:

In [ ]:
# post processing
ds["lon"] = ds["lon"] % 360
ds["lon"] = ds["lon"].where(ds["lon"] <= 180, ds["lon"] - 360)

In [ ]:
# with a Kernel
def periodicBC(particles, fieldset):  # pragma: no cover
    particles.dlon = np.where(
        particles.lon + particles.dlon > 180, particles.dlon - 360, particles.dlon
    )


pset = parcels.ParticleSet(fieldset, lon=lonp, lat=latp)
pfile = parcels.ParticleFile(
    store="output_curvilinear_periodic.zarr", outputdt=np.timedelta64(1, "D")
)

pset.execute(
    [parcels.kernels.AdvectionEE, periodicBC],
    runtime=runtime,
    dt=np.timedelta64(1, "D"),
    output_file=pfile,
)

In [ ]:
ds_periodic = xr.open_zarr("output_curvilinear_periodic.zarr")

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(ds.lon.T, ds.lat.T, ".-")
ax[0].vlines(np.arange(-180, 360, 360), -90, 90, color="r", label="antimeridian")
ax[0].set_ylabel("Latitude [deg N]")
ax[0].set_xlabel("Longitude [deg E]")
ax[0].set_xticks(np.arange(-180, 181, 45))
ax[0].set_title("in post processing")
ax[0].legend(loc="lower center")

ax[1].plot(ds_periodic.lon.T, ds_periodic.lat.T, ".-")
ax[1].vlines(np.arange(-180, 360, 360), -90, 90, color="r", label="antimeridian")
ax[1].set_ylabel("Latitude [deg N]")
ax[1].set_xlabel("Longitude [deg E]")
ax[1].set_xticks(np.arange(-180, 181, 45))
ax[1].set_title("with periodic Kernel")
ax[1].legend(loc="lower center")

plt.show()

## Running 3D Nemo simulations

Here, we focus on 3D fields. Basically, it is a straightforward extension of the 2D example above.

We again use the combination of the `parcels.convert.nemo_to_sgrid` function and the `parcels.FieldSet.from_sgrid_conventions()` to create a FieldSet object with the correct `Grid` information for the NEMO data.

In [ ]:
data_folder = parcels.download_example_dataset("NemoNorthSeaORCA025-N006_data")
ds_fields = xr.open_mfdataset(
    data_folder.glob("ORCA*.nc"),
    data_vars="minimal",
    coords="minimal",
    compat="override",
)
ds_coords = xr.open_dataset(data_folder / "coordinates.nc", decode_times=False)
ds_fset = parcels.convert.nemo_to_sgrid(
    fields={"U": ds_fields["uo"], "V": ds_fields["vo"], "W": ds_fields["wo"]},
    coords=ds_coords,
)
fieldset = parcels.FieldSet.from_sgrid_conventions(ds_fset)

The code below is an example of how to then create a 3D simulation with particles, starting on a meridional line through the North Sea from the mouth of the river Rhine at 100m depth, and advecting them through the North Sea using the `AdvectionRK2_3D`

In [ ]:
npart = 10
pset = parcels.ParticleSet(
    fieldset=fieldset,
    lon=np.linspace(1.9, 3.4, npart),
    lat=np.linspace(65, 51.6, npart),
    z=100 * np.ones(npart),
)

pfile = parcels.ParticleFile(
    store="output_nemo3D.zarr", outputdt=np.timedelta64(1, "D")
)

pset.execute(
    parcels.kernels.AdvectionRK2_3D,
    endtime=fieldset.time_interval.right,
    dt=np.timedelta64(6, "h"),
    output_file=pfile,
)

We can then plot the trajectories on top of the surface U field

In [ ]:
field = fieldset.U.data[0, 0, :, :]
field = field.where(field != 0, np.nan)  # Mask land values for better plotting
plt.pcolormesh(fieldset.U.grid.lon, fieldset.U.grid.lat, field, cmap="RdBu")

ds_out = xr.open_zarr("output_nemo3D.zarr")
plt.scatter(ds_out.lon.T, ds_out.lat.T, c=-ds_out.z.T, marker=".")
plt.colorbar(label="Depth (m)")
plt.show()